In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import random
import joblib
import os
os.chdir('Resources/')

from custom_tuple_scaler import CustomTupleScaler

In [2]:
df = pd.read_csv('6_CC_Encrypted_Data.csv')

In [3]:
Y = df[['HeartDisease']]
Y

,HeartDisease
0,0
1,1
2,0
3,1
4,0
...,...
3245,1
3246,1
3247,0
3248,0


In [4]:
c1_list = []
c2_list = []

for k in range(11):
    cnt = 0
    c1_sum = 0
    c2_sum = 0
    
    messages = df.iloc[:, k].values

    for i, message in enumerate(messages, 1):
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))
            c1_sum += c1
            c2_sum += c2
            cnt += 1
    
    print(c1_sum//cnt, ",", c2_sum//cnt)

    c1_list.append(c1_sum//cnt)
    c2_list.append(c2_sum//cnt)

print(c1_list)
print(c2_list)

6965361356960251970681699416527167003396930684399 , 30217139673302031254829592992378024533878940689657
54600078953351133152003341118517817314167801601939 , 23882417530828833820232030289434178573895492614734
36649207273954309735937122332365840930023873680884 , 31257445741747527543331712533634722199901749334179
83860246762706255172761784370775066461479861118299 , 47736326788307872921021879683387002044728535726667
16043955655060611551799098102694158622740324518298 , 43570977182708344300493988886736242425372723276623
48796268997449217309075469418552965717916389030766 , 21270487134087081655449550974215752004786739471885
3857767734957454709824171872090418722115974714484 , 6007744126266429215264072859610897495757862371132
83297904217885534795359347609074572793018462457138 , 40148136876669348027076612145139127910533037019312
12956688826076395746654158514081595251862400502558 , 5473654934117652083434044599580847510549404051408
5759314526113684676869960964229505210431962579413 , 9488133877349612

In [5]:
column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol",
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina",
    "Oldpeak", "ST_Slope", "HeartDisease"
]

In [6]:
imputed_data = pd.DataFrame()

for k in range(11):
    
    messages = df.iloc[:, k].values

    imputed_column = []

    for i, message in enumerate(messages, 1):
        if message is None:
            c1 = c1_list[k]
            c2 = c2_list[k]
        
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))

        imputed_column.append(f"({c1},{c2})")

    imputed_data[column_names[k]] = imputed_column

In [7]:
scaler = joblib.load('13_CC_Scaler.joblib')

In [8]:
df_scaled = scaler.transform(imputed_data)
df_scaled = pd.DataFrame(df_scaled, columns=imputed_data.columns)
# df_scaled.to_csv('scaled_new_data.csv', index=False)

g:\Thesis - Copy\Resources\custom_tuple_scaler.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  c2_values = X.applymap(self._extract_c2)


In [9]:
with open("12_SP_Test_Data_RF_IF.txt", "r") as file:
    test_indices = eval(file.read()) 

In [10]:
file_path = "13_CC_F1_Scores_RF_IF.txt"

with open(file_path, "r") as f:
    lines = f.readlines()

reference_f1_scores = []
for line in lines:
    if "F1-score for fold" in line:
        score = float(line.split(":")[1].strip())
        reference_f1_scores.append(score)

In [22]:
fold_to_test = random.randint(1, 5)

test_indices_fold = test_indices[fold_to_test - 1] 
print(f"Test indices for fold {fold_to_test}: {test_indices_fold}")

X_test = df_scaled.iloc[test_indices_fold]

model_path = f'12_SP_Model_RF_IF_{fold_to_test}.joblib'
model = joblib.load(model_path)

Y_test = Y.iloc[test_indices_fold]
Y_pred = model.predict(X_test)

computed_f1_fold = f1_score(Y_test, Y_pred) * 100

reference_f1_fold = reference_f1_scores[fold_to_test - 1]

print(f"Computed F1-score for fold 1: {computed_f1_fold:.2f}")
print(f"Reference F1-score for fold 1: {reference_f1_fold:.2f}")

if np.isclose(computed_f1_fold, reference_f1_fold, atol=1e-2):
    print("The computed F1-score matches the reference score.")
else:
    print("The computed F1-score does not match the reference score.")

Test indices for fold 2: [4, 12, 14, 15, 16, 35, 40, 41, 42, 45, 52, 57, 59, 62, 63, 67, 69, 73, 74, 75, 82, 95, 103, 117, 132, 138, 140, 153, 154, 157, 163, 179, 183, 189, 191, 193, 201, 202, 207, 214, 219, 222, 223, 225, 230, 231, 235, 241, 251, 252, 256, 258, 265, 270, 279, 287, 307, 328, 334, 337, 343, 348, 352, 358, 362, 364, 373, 374, 377, 394, 401, 405, 412, 417, 418, 430, 431, 434, 435, 438, 440, 446, 453, 457, 461, 463, 468, 477, 481, 490, 493, 499, 508, 513, 517, 518, 519, 520, 525, 528, 533, 536, 537, 541, 553, 556, 557, 559, 561, 562, 564, 577, 582, 583, 584, 586, 587, 595, 596, 600, 603, 608, 609, 615, 618, 619, 620, 627, 630, 634, 644, 648, 652, 654, 658, 666, 667, 674, 683, 684, 687, 696, 698, 700, 706, 727, 728, 731, 736, 740, 742, 748, 750, 752, 759, 761, 763, 765, 766, 767, 777, 791, 798, 799, 801, 805, 808, 810, 813, 815, 817, 823, 829, 836, 844, 845, 858, 860, 862, 863, 866, 867, 873, 879, 902, 906, 910, 911, 917, 921, 931, 940, 942, 954, 957, 959, 961, 965, 968, 98